# Homework 2
#### Name: Syed Zain Raza
#### CWID: 20011917

In [5]:
# optional: allow Jupyter to "hot reload" the Python modules I wrote, to avoid restarting the kernel after every change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Problem 1: Teddy Stereo

### Part 1: Loading the Images

In [2]:
from util import ops

In [3]:
left_img = ops.load_image(
    "./teddy/teddyL.pgm",
    return_grayscale=True,
    return_array=True,
)

Dimensions of ./teddy/teddyL.pgm: 375 x 450


In [4]:
right_img = ops.load_image(
    "./teddy/teddyR.pgm",
    return_grayscale=True,
    return_array=True,
)

Dimensions of ./teddy/teddyR.pgm: 375 x 450


### Part 2: Preprocessing

#### Using the Rank Transform

In [ ]:
import numpy as np

class RankTransform2D:
    @staticmethod
    def transform(
        image: np.ndarray,
        filter_size: int,
        do_logging: bool = False,
    ) -> np.ndarray:
        """
        Perform a rank filtering operation on an image.

        The goal is to produce a new image where each cell value
        represents the "rank" of the corresponding pixel in the input
        (i.e., the index of said pixel in a sorted list of itself &
        the neighboring pixel values).

        Parameters:
            image(np.ndarray): in case its RGB, the transform will be
                                per channel. Please pass the image in
                                channels-last format.
            filter_size(int): this is k. The size of each local neighborhood
                              will be kxk. Please pass an odd value > 0.
        
        Returns: np.ndarray: the transformed image
        """
        ### HELPER(S)
        def compute_rank(
            channel: np.ndarray,
            kernel: np.ndarray,
            row_index: int,
            col_index: int,
        ) -> float:
            """
            Computes the rank of 1 local window of the image.

            Parameters:
                channel(array-like): one of the channels of the input image
                kernel(array-like): tells us the size of the window 
                row_index, col_index: int: the coordinates of the upper left corner
                                            of the block of pixels being ranked

            Returns: int: the rank of the center pixel of the window
            """
            # A: define useful vars
            kernel_h, kernel_w = kernel.shape
            # B: get the block of pixels needed for the convolution
            block_of_pixels = channel[
                row_index : (kernel_h + row_index),
                col_index : (kernel_w + col_index)
            ]
            # C: count the of # higher than the center
            center_val = block_of_pixels[kernel_h // 2, kernel_w // 2]
            if do_logging:
                print(f"I think that {center_val} is at the center of {block_of_pixels}")
            transformed_block = np.where(block_of_pixels < center_val, 1, 0)
            if do_logging:
                print(f"Transformed bloc <{block_of_pixels}> into: <{transformed_block}>")
            return np.sum(transformed_block)

        ### DRIVER
        # data validation
        assert isinstance(image, np.ndarray) 
        assert image.shape > (0, 0)
        assert isinstance(filter_size, int)
        assert filter_size > 0 and filter_size % 2 == 1

        # make a copy of the img, padded - will be an intermediate repr
        kernel = np.ones((filter_size, filter_size))
        num_channels = -1
        if len(image.shape) == 2:  # grayscale
            num_channels = 1
            padded_image, _, _ = ops.pad(
                image, kernel, stride=1, padding_type="zero"
            )
        elif len(image.shape) == 3:  # RGB
            num_channels = image.shape[2]
            channels = [
                ops.pad(image[:, :, channel_index], kernel, stride=1, padding_type="zero")[0]
                for channel_index in range(num_channels)
            ]
            padded_image = np.dstack(channel)

        # fill in the output
        stride = 1
        output_image = list()
        for image_channel_index in np.arange(num_channels):
            transformed_channel = list()
            channel = (
                padded_image[:, :, image_channel_index]
                if num_channels > 1
                else padded_image
            )
            kernel_h, _ = kernel.shape
            # iterate over the rows and columns
            starting_row_ndx = 0
            while starting_row_ndx <= len(channel) - kernel_h:
                # convolve the next row of this channel
                next_channel_row = ops.slide_kernel_over_image(
                    channel,
                    kernel,
                    starting_row_ndx,
                    stride,
                    apply=compute_rank,
                )
                # now, add the convolved row to the list
                transformed_channel.append(next_channel_row)
                # move to the next starting row for the filtering
                starting_row_ndx += stride
            output_image.append(transformed_channel)
        # stack the channels, and return
        return np.dstack(output_image)